In [1]:
import os
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import imageio

In [20]:
# Function to plot each month's data for a given radiation variable
def plot_monthly_radiation(variable_name, time_index, radiation_data, lat, lon, output_folder):
    lon2d, lat2d = np.meshgrid(lon, lat)
    fig = plt.figure(figsize=(10, 7))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines()
    ax.set_global()

    # Determine color scale limits based on the variable being plotted
    if "net" in variable_name:
        vmin, vmax = -450, 450  # For net radiation, we use a range from -450 to 450
        cmap = 'seismic'
    else:
        vmin, vmax = 0, 450  # For shortwave and longwave, we use a range from 0 to 450
        cmap = 'magma'

    # Create the color mesh with the determined scale and colormap
    radiation_plot = ax.pcolormesh(lon2d, lat2d, radiation_data[time_index, :, :], cmap=cmap, vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree())
    cbar = plt.colorbar(radiation_plot, orientation='vertical', shrink=0.7)
    cbar.set_label('Radiation (W/m^2)')

    ax.gridlines(draw_labels=True)
    plt.title(f'{variable_name} {time_index//12 + 2000}-{time_index%12 + 1:02d}')

    # Create the output directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Save the figure
    plt.savefig(os.path.join(output_folder, f'{variable_name}_{time_index//12 + 2000}_{time_index%12 + 1:02d}.png'), dpi=300)
    
    # Close the plot to save memory
    plt.close(fig)

# Function to create an animation from plots
def create_animation(variable_name, output_folder, image_folder, frame_duration):
    images = []
    for file_name in sorted(os.listdir(image_folder)):
        if file_name.startswith(variable_name) and file_name.endswith('.png'):
            file_path = os.path.join(image_folder, file_name)
            images.append(imageio.imread(file_path))
    output_gif_path = os.path.join(output_folder, f'{variable_name}_Animation.gif')
    imageio.mimsave(output_gif_path, images, duration=frame_duration)
    print(f"Animated GIF for {variable_name} saved as {output_gif_path}")

# Main function to process the data and create animations
def process_and_create_animations(variable_names, dataset_path, output_folder, frame_duration=0.5):
    data = nc.Dataset(dataset_path)
    lat = data.variables['lat'][:]
    lon = data.variables['lon'][:]
    os.makedirs(output_folder, exist_ok=True)
    
    for variable_name in variable_names:
        radiation_variable = data.variables[variable_name][:]
        for time_index in range(radiation_variable.shape[0]):
            plot_monthly_radiation(variable_name, time_index, radiation_variable, lat, lon, output_folder)
        create_animation(variable_name, output_folder, output_folder, frame_duration)

    data.close()

# Define variables and dataset path
variable_names = ['toa_sw_all_mon', 'toa_lw_all_mon', 'toa_net_all_mon']
dataset_path = 'data/CERES_EBAF-TOA_Ed4.2_Subset_200003-202308.nc'
output_folder = 'CERES_plots'

# Call the main function
#This takes about 10 minutes to run for the entire dataset
process_and_create_animations(variable_names, dataset_path, output_folder, frame_duration=0.5)

/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/998788205.py:53: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  radiation_variable = data.variables[variable_name][:]
/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/998788205.py:53: UserWarning: WARNING: valid_max not used since it
cannot be safely cast to variable data type
  radiation_variable = data.variables[variable_name][:]
/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/998788205.py:40: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))


Animated GIF for toa_sw_all_mon saved as CERES_plots/toa_sw_all_mon_Animation.gif


/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/998788205.py:53: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  radiation_variable = data.variables[variable_name][:]
/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/998788205.py:53: UserWarning: WARNING: valid_max not used since it
cannot be safely cast to variable data type
  radiation_variable = data.variables[variable_name][:]
/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/998788205.py:40: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))


Animated GIF for toa_lw_all_mon saved as CERES_plots/toa_lw_all_mon_Animation.gif


/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/998788205.py:53: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  radiation_variable = data.variables[variable_name][:]
/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/998788205.py:53: UserWarning: WARNING: valid_max not used since it
cannot be safely cast to variable data type
  radiation_variable = data.variables[variable_name][:]
/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/998788205.py:40: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))


Animated GIF for toa_net_all_mon saved as CERES_plots/toa_net_all_mon_Animation.gif


In [25]:
def plot_monthly_radiation(variable_name, time_index, radiation_data, lat, lon, output_folder):
    lon2d, lat2d = np.meshgrid(lon, lat)

    # Define the rotation for each time index
    # Longitude rotates 10 degrees per month
    rotation_longitude = (time_index * 10) % 360
    # Latitude oscillates between -45 and 45 degrees over the course of a year (12 months)
    # using a sine function that oscillates between -1 and 1, scaled and shifted to fit the -45 to 45 range
    rotation_latitude = 45

    fig = plt.figure(figsize=(10, 7))
    # Use Orthographic projection for spherical appearance
    ax = plt.axes(projection=ccrs.Orthographic(central_longitude=rotation_longitude, central_latitude=rotation_latitude))
    ax.coastlines()
    ax.set_global()

    # Determine color scale limits based on the variable being plotted
    if "net" in variable_name:
        vmin, vmax = -450, 450  # For net radiation, we use a range from -450 to 450
        cmap = 'seismic'
    else:
        vmin, vmax = 0, 450  # For shortwave and longwave, we use a range from 0 to 450
        cmap = 'magma'

    # Create the color mesh with the determined scale and colormap
    radiation_plot = ax.pcolormesh(lon2d, lat2d, radiation_data[time_index, :, :], cmap=cmap, vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree())
    cbar = plt.colorbar(radiation_plot, orientation='vertical', shrink=0.7)
    cbar.set_label('Radiation (W/m^2)')

    plt.title(f'{variable_name} {time_index//12 + 2000}-{time_index%12 + 1:02d}')

    # Create the output directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Save the figure
    plt.savefig(os.path.join(output_folder, f'{variable_name}_{time_index//12 + 2000}_{time_index%12 + 1:02d}.png'), dpi=300)
    
    # Close the plot to save memory
    plt.close(fig)

# Function to create an animation from plots
def create_animation(variable_name, output_folder, image_folder, frame_duration):
    images = []
    for file_name in sorted(os.listdir(image_folder)):
        if file_name.startswith(variable_name) and file_name.endswith('.png'):
            file_path = os.path.join(image_folder, file_name)
            images.append(imageio.imread(file_path))
    output_gif_path = os.path.join(output_folder, f'{variable_name}_Animation.gif')
    imageio.mimsave(output_gif_path, images, duration=frame_duration)
    print(f"Animated GIF for {variable_name} saved as {output_gif_path}")

# Main function to process the data and create animations
def process_and_create_animations(variable_names, dataset_path, output_folder, frame_duration=0.5):
    data = nc.Dataset(dataset_path)
    lat = data.variables['lat'][:]
    lon = data.variables['lon'][:]
    os.makedirs(output_folder, exist_ok=True)
    
    for variable_name in variable_names:
        radiation_variable = data.variables[variable_name][:]
        for time_index in range(radiation_variable.shape[0]):
            plot_monthly_radiation(variable_name, time_index, radiation_variable, lat, lon, output_folder)
        create_animation(variable_name, output_folder, output_folder, frame_duration)

    data.close()

# Define variables and dataset path
variable_names = ['toa_sw_all_mon', 'toa_lw_all_mon', 'toa_net_all_mon']
dataset_path = 'data/CERES_EBAF-TOA_Ed4.2_Subset_200003-202308.nc'
output_folder = 'CERES_plots'

# Call the main function
process_and_create_animations(variable_names, dataset_path, output_folder, frame_duration=0.5)

/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/1125026258.py:60: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  radiation_variable = data.variables[variable_name][:]
/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/1125026258.py:60: UserWarning: WARNING: valid_max not used since it
cannot be safely cast to variable data type
  radiation_variable = data.variables[variable_name][:]
/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/1125026258.py:47: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))


Animated GIF for toa_sw_all_mon saved as CERES_plots/toa_sw_all_mon_Animation.gif


/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/1125026258.py:60: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  radiation_variable = data.variables[variable_name][:]
/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/1125026258.py:60: UserWarning: WARNING: valid_max not used since it
cannot be safely cast to variable data type
  radiation_variable = data.variables[variable_name][:]
/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/1125026258.py:47: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))


Animated GIF for toa_lw_all_mon saved as CERES_plots/toa_lw_all_mon_Animation.gif


/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/1125026258.py:60: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  radiation_variable = data.variables[variable_name][:]
/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/1125026258.py:60: UserWarning: WARNING: valid_max not used since it
cannot be safely cast to variable data type
  radiation_variable = data.variables[variable_name][:]
/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/1125026258.py:47: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))


Animated GIF for toa_net_all_mon saved as CERES_plots/toa_net_all_mon_Animation.gif


In [2]:
import netCDF4 as nc

def check_variables_in_dataset(dataset_path):
    data = nc.Dataset(dataset_path)
    print("Available variables in the dataset:")
    for variable in data.variables:
        print(variable)
        print("explains", data.variables[variable].long_name)
    data.close()

dataset_path = 'data/CERES_SYN1deg-Day_Terra-Aqua-MODIS_Ed4.1_Subset_20230501-20230630.nc'
check_variables_in_dataset(dataset_path)


Available variables in the dataset:
time
explains Time
lon
explains Longitude
lat
explains Latitude
adj_toa_sw_down_band_sw1_7_daily
explains Adjusted All-Sky Spectral Shortwave Fluxes All-Sky TOA Spectral Shortwave Down Flux, SW1-7 band, Daily Means
adj_toa_sw_down_band_sw8_10_daily
explains Adjusted All-Sky Spectral Shortwave Fluxes All-Sky TOA Spectral Shortwave Down Flux, SW8-10 band, Daily Means
adj_toa_sw_down_band_sw11_13_daily
explains Adjusted All-Sky Spectral Shortwave Fluxes All-Sky TOA Spectral Shortwave Down Flux, SW11-13 band, Daily Means
adj_toa_sw_down_band_sw14_18_daily
explains Adjusted All-Sky Spectral Shortwave Fluxes All-Sky TOA Spectral Shortwave Down Flux, SW14-18 band, Daily Means
adj_toa_sw_up_band_sw1_7_daily
explains Adjusted All-Sky Spectral Shortwave Fluxes All-Sky TOA Spectral Shortwave Up Flux, SW1-7 band, Daily Means
adj_toa_sw_up_band_sw8_10_daily
explains Adjusted All-Sky Spectral Shortwave Fluxes All-Sky TOA Spectral Shortwave Up Flux, SW8-10 band, Da

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import os
import netCDF4 as nc
import imageio

# Modified plot function for daily radiation
def plot_daily_radiation(variable_name, time_index, radiation_data, lat, lon, output_folder, time_units, calendar):
    lon2d, lat2d = np.meshgrid(lon, lat)

    # Define the rotation for each time index
    # Since we're dealing with daily data, let's say we rotate 1 degree per day
    rotation_longitude = (time_index * 5) % 360
    # Keeping the latitude fixed for simplicity
    rotation_latitude = 25

    fig = plt.figure(figsize=(10, 7))
    # Use Orthographic projection for spherical appearance with rotation
    ax = plt.axes(projection=ccrs.Orthographic(central_longitude=rotation_longitude, central_latitude=rotation_latitude))
    ax.coastlines()
    ax.set_global()

    vmin, vmax = 0, 450  # Assuming a range from 0 to 450 for visualization
    cmap = 'magma'

    radiation_plot = ax.pcolormesh(lon2d, lat2d, radiation_data[time_index, :, :], cmap=cmap, vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree())
    cbar = plt.colorbar(radiation_plot, orientation='vertical', shrink=0.7)
    cbar.set_label('Radiation (W/m^2)')

    date_str = nc.num2date(time_index, units=time_units, calendar=calendar)
    plt.title(f'{variable_name} {date_str.strftime("%Y-%m-%d")}')

    os.makedirs(output_folder, exist_ok=True)
    plt.savefig(os.path.join(output_folder, f'{variable_name}_{date_str.strftime("%Y%m%d")}.png'), dpi=300)
    plt.close(fig)



def create_animation(variable_name, output_folder, image_folder, frame_duration):
    images = []
    for file_name in sorted(os.listdir(image_folder)):
        if file_name.startswith(variable_name) and file_name.endswith('.png'):
            file_path = os.path.join(image_folder, file_name)
            images.append(imageio.imread(file_path))
    output_gif_path = os.path.join(output_folder, f'{variable_name}_Animation.gif')
    imageio.mimsave(output_gif_path, images, duration=frame_duration)
    print(f"Animated GIF for {variable_name} saved as {output_gif_path}")

#  process and create animations for daily data
def process_and_create_daily_animations(variable_names, dataset_path, output_folder, frame_duration=0.5):
    data = nc.Dataset(dataset_path)
    lat = data.variables['lat'][:]
    lon = data.variables['lon'][:]
    time_units = data.variables['time'].units
    calendar = data.variables['time'].calendar if 'calendar' in data.variables['time'].ncattrs() else 'standard'
    
    os.makedirs(output_folder, exist_ok=True)

    for variable_name in variable_names:
        radiation_variable = np.sum([data.variables[v][:] for v in variable_names[variable_name]], axis=0)
        for time_index in range(radiation_variable.shape[0]):
            plot_daily_radiation(variable_name, time_index, radiation_variable, lat, lon, output_folder, time_units, calendar)
        create_animation(variable_name, output_folder, output_folder, frame_duration)

    data.close()

# Define new variable groups for daily data
variable_names = {
    'total_daily_outgoing_sw': ['adj_toa_sw_up_band_sw1_7_daily', 'adj_toa_sw_up_band_sw8_10_daily', 'adj_toa_sw_up_band_sw11_13_daily', 'adj_toa_sw_up_band_sw14_18_daily'],
    'total_daily_outgoing_lw': ['adj_toa_lw_up_band_lw1_4_daily', 'adj_toa_lw_up_band_lw5_7_daily', 'adj_toa_lw_up_band_lw8_9_daily', 'adj_toa_lw_up_band_lw10_11_daily', 'adj_toa_lw_up_band_lw12_daily']
}

dataset_path = 'data/CERES_SYN1deg-Day_Terra-Aqua-MODIS_Ed4.1_Subset_20230501-20230630.nc'
output_folder = 'CERES_daily_plots'

# Call the updated main function
process_and_create_daily_animations(variable_names, dataset_path, output_folder, frame_duration=0.5)


/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_84193/3519466016.py:61: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  radiation_variable = np.sum([data.variables[v][:] for v in variable_names[variable_name]], axis=0)
/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_84193/3519466016.py:61: UserWarning: WARNING: valid_max not used since it
cannot be safely cast to variable data type
  radiation_variable = np.sum([data.variables[v][:] for v in variable_names[variable_name]], axis=0)
/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_84193/3519466016.py:45: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))


Animated GIF for total_daily_outgoing_sw saved as CERES_daily_plots/total_daily_outgoing_sw_Animation.gif


/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_84193/3519466016.py:61: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  radiation_variable = np.sum([data.variables[v][:] for v in variable_names[variable_name]], axis=0)
/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_84193/3519466016.py:61: UserWarning: WARNING: valid_max not used since it
cannot be safely cast to variable data type
  radiation_variable = np.sum([data.variables[v][:] for v in variable_names[variable_name]], axis=0)
/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_84193/3519466016.py:45: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))


Animated GIF for total_daily_outgoing_lw saved as CERES_daily_plots/total_daily_outgoing_lw_Animation.gif
